In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from geneticalgorithm import geneticalgorithm as ga

In [2]:
def formular(X, Imsc, ns, npp, I0, A):
    S, T = X
    # 常数
    k = 1.38 * 1e-23
    # A = 1
    q = 1.6 * 1e-19
    # 公式
    Im= Imsc * S / 1000 + 1.6 * 1e-3 * (T - 298)
    Rs = ns * A * k * T / q * (np.log((npp * Im - Im + npp * I0) / npp * I0) + (npp-1)*npp*I0*Im / ((npp-1)*Im+npp*I0)) / (2 * Im)
    Um = ns * A * k * T / q * np.log((npp * Im - Im + npp * I0) / npp * I0) - Im * Rs
    Im = np.abs(Im)
    Um = np.abs(Um)
    Pm = Im * Um
    return Pm

In [3]:
i=1
train = pd.read_csv('D:/dissertation/dataset/final_dataset/train_{i}.csv'.format(i=i))
train.shape

(53294, 172)

In [4]:
train_ = train[
    (train['实发辐照度'] >=0) & 
    (train['实发辐照度'].notna() &
     train['实际功率']>0)
]
S = train_['实发辐照度']
T = train_['温度.1']
T = T + 273
power = train_['实际功率']

In [ ]:
def objective(params):
    Imsc, ns, npp, I0, A = params
    Pm_pred = formular((S, T), Imsc, ns, npp, I0, A)
    return np.mean((Pm_pred - power) ** 2)

# # train1,2,3,
# varbounds = np.array([
#     [0, 15],            # Imsc
#     [5, 10],           # ns
#     [1, 5],             # npp
#     [1e-4, 1e-2],      # I0
#     [1, 2]              # A
# ])

# train4
varbounds = np.array([
    [0, 20],            # Imsc
    [5, 10],           # ns
    [1, 5],             # npp
    [1e-4, 1e-2],      # I0
    [1, 2]              # A
])

model = ga(function=objective, dimension=5, variable_type='real', variable_boundaries=varbounds, 
           algorithm_parameters={'max_num_iteration': 100,  # 增加最大迭代次数
                                'population_size':100,    # 增大种群规模（默认可能为50）   
                                'mutation_probability':0.2,  # 变异概率（默认0.1，可尝试0.05~0.2）
                                'elit_ratio':0.01,        # 保留精英个体的比例
                                'crossover_probability':0.7,  # 交叉概率（通常0.7~0.9）
                                'parents_portion':0.3,    # 选择父代的比例
                                'crossover_type':'one_point',  # 交叉方式（还可选'one_point'/'two_point'）
                                'max_iteration_without_improv':None
                                })
model.run()
Imsc, ns, npp, I0, A = model.output_dict['variable']

In [ ]:
# if i == 1:
#     Imsc, ns, npp, I0, A = 1.35820600e+01, 9.03559829e+00, 1.19937217e+00, 9.66981160e-03, 1.44747693e+00
# elif i==2: 
#     Imsc, ns, npp, I0, A = 1.49075359e+01, 9.61505593e+00, 2.81886732e+00,  8.26063407e-03, 1.91213751e+00
# elif i==3: 
#     Imsc, ns, npp, I0, A = 1.48077387e+01, 9.93317152e+00, 1.00501179e+00, 2.51756279e-04, 1.93719431e+00
# elif i==4: 
#     Imsc, ns, npp, I0, A = 1.97104746e+01, 9.54997229e+00, 1.00316405e+00, 1.14406498e-03, 1.96981975e+00

In [ ]:
train['pred_power_cell'] = 0
train.loc[train['实发辐照度'] > 0 & train['实发辐照度'].notna(), 'pred_power_cell'] = formular(
    (train['实发辐照度'], train['温度.1']+273), Imsc, ns, npp, I0, A
)
train['pred_power_cell'] = train['pred_power_cell'].fillna(0)
# train = train.drop('温度.1', axis=1)
train.to_csv('D:/dissertation/dataset/final_dataset/train_{i}.csv'.format(i=i), index=False)
train.shape

In [8]:
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error
def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

In [ ]:
# mse
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(train['pred_power_cell'], train['实际功率'])

mse = mean_squared_error(train['实际功率'], train['pred_power_cell'])
mae = mean_absolute_error(train['实际功率'], train['pred_power_cell'])
rmse = rmse(train['实际功率'], train['pred_power_cell'])
print(f'MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

In [10]:
# test
test = pd.read_csv('../dataset/final_dataset/test_{i}.csv'.format(i=i)) 
test['pred_power_cell'] = 0  # 初始化为0

In [11]:
# 对于'实发辐照度'大于0的行，使用formular函数计算实际功率
test.loc[test['实发辐照度'] > 0, 'pred_power_cell'] = formular(
    (test['实发辐照度'], test['温度.1']+273), Imsc, ns, npp, I0, A
)
test['pred_power_cell'] = test['pred_power_cell'].fillna(0)
# test = test.drop('温度.1', axis=1)
test.shape

d:\anaconda3\envs\dissertation\lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


(13372, 172)

In [12]:
# mse
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(test['实际功率'], test['pred_power_cell'])
mae = mean_absolute_error(test['实际功率'], test['pred_power_cell'])
rmse = rmse(test['实际功率'], test['pred_power_cell'])
print(f'MSE: {mse:.4f}, MAE: {mae:.4f}, RMSE: {rmse:.4f}')

MSE: 1.6253, MAE: 0.8220, RMSE: 1.2749


In [ ]:
test.to_csv('D:/dissertation/dataset/final_dataset/test_{i}.csv'.format(i=i), index=False)